Capítulo 6

6.4 Patrones de unión.

6.4.3 Unión externa 

6.4.3.1 Unión externa por la izquierda

Patrón que permite realizar una unión por la izquierda de dos tablas de datos


In [1]:
%%writefile unionIzquierda.py
#!/usr/bin/env python
from mrjob.job import MRJob
import re,sys,os

class unionIzquierda(MRJob):
    
    def limpiarNombreArchivo(self,archivo):
        encontradaBarra=False
        tamano=len(archivo)
        posicion=tamano-1
        while encontradaBarra==False or posicion==0:
            if archivo[posicion]=="/":
                encontradaBarra=True
                return archivo[posicion+1:tamano]
            else:
                posicion-=1
        if posicion==0:
            return archivo
        
    def mapper_init(self):
        self.namefile=self.limpiarNombreArchivo(os.getenv('map_input_file')) 
        
    def mapper(self,_,line):
        clave=""
        linea=line.split(';')
        encontrado=re.search('[a-zA-Z]',linea[0])
        if encontrado==None:
            if self.namefile=="tiendas.csv":
                linea.append(self.namefile)
                clave=linea[0] #Esta clave es la común de la tabla 1
                yield clave,linea
            else:
                linea.append('art.csv') #se recorta el nombre del archivo para mejorar la legibilidad de la salida
                clave=linea[1] #Clave común de la tabla 2
                yield clave,linea
       
    ############# Reducer ############################
        
    def reducer(self,key,values):
        listaA=[]
        listaB=[]
        #Llenamos las dos listas
        for valor in values:
            if valor[len(valor)-1]=="tiendas.csv":
                listaA.append(valor)
            else:
                listaB.append(valor)
        
        ###### Union por la izquierda ######
        for valorA in listaA:
            if listaB:
                for valorB in listaB:
                    yield valorA, valorB
            else:
                #Si la listaB está vacía
                yield valorA, "null" 
                
if __name__ == '__main__':
    unionIzquierda.run()

Writing unionIzquierda.py


In [2]:
!python unionIzquierda.py archivos_datos/tiendas-articulos/tiendas.csv archivos_datos/tiendas-articulos/articulos_stock.csv

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/unionIzquierda.root.20201214.221444.910643
Running step 1 of 1...
job output is in /tmp/unionIzquierda.root.20201214.221444.910643/output
Streaming final output from /tmp/unionIzquierda.root.20201214.221444.910643/output...
["3", "Granada", "tiendas.csv"]	["1006", "3", "3", "art.csv"]
["3", "Granada", "tiendas.csv"]	["1010", "3", "20", "art.csv"]
["6", "Barcelona", "tiendas.csv"]	"null"
["2", "Albacete", "tiendas.csv"]	["1002", "2", "10", "art.csv"]
["2", "Albacete", "tiendas.csv"]	["1005", "2", "5", "art.csv"]
["2", "Albacete", "tiendas.csv"]	["1008", "2", "5", "art.csv"]
["1", "Santander", "tiendas.csv"]	["1001", "1", "20", "art.csv"]
["1", "Santander", "tiendas.csv"]	["1003", "1", "15", "art.csv"]
["1", "Santander", "tiendas.csv"]	["1007", "1", "8", "art.csv"]
Removing temp directory /tmp/unionIzquierda.root.20201214.221444.910643...


EJECUCIÓN EN EL CLUSTER:

In [1]:
!python unionIzquierda.py hdfs:///archivos_datos/articulos_stock.csv hdfs:///archivos_datos/tiendas.csv -r hadoop --python-bin /opt/anaconda/bin/python3.7

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /usr/lib/hadoop/bin...
Found hadoop binary: /usr/lib/hadoop/bin/hadoop
Using Hadoop version 2.6.0
Looking for Hadoop streaming jar in /usr/lib/hadoop...
Looking for Hadoop streaming jar in /usr/lib/hadoop-mapreduce...
Found Hadoop streaming jar: /usr/lib/hadoop-mapreduce/hadoop-streaming.jar
Creating temp directory /tmp/unionIzquierda.root.20201214.223814.456168
uploading working dir files to hdfs:///user/root/tmp/mrjob/unionIzquierda.root.20201214.223814.456168/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/unionIzquierda.root.20201214.223814.456168/files/
Running step 1 of 1...
  packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob7549110537948998925.jar tmpDir=null
  Connecting to ResourceManager at yarnmaster/172.18.0.5:8032
  Connecting to ResourceManager at yarnmaster/172.18.0.5:8032
  Total inp